In [2]:
def draw_multiplex_network_communities(layers,k_):
    
    layer12_concat = pd.concat(layers,axis=0)
    nodes, edges = network_reformat(layer12_concat, weighted=True, output_format='graph',node_format="hash")
    
    hash_to_int = dict(zip(nodes,range(len(nodes))))
    int_to_hash = dict(zip(range(len(nodes)),nodes))
    
    G = nx.Graph()
    G.add_nodes_from([hash_to_int[n] for n in nodes])
    G.add_weighted_edges_from([(hash_to_int[i],hash_to_int[j], w) for i,j,w in edges])
    #nx.set_node_attributes(G, 'community', node_communities)
    
    
    node_communities = [set([hash_to_int[n] for n in c]) for c in community_detection_multiplex(*layers)]
    
    
    return node_communities
    
    
    
    return nx.get_node_attributes(G, 'community')
    
    # Initialise high-level graph
    G_communities = nx.Graph()

    community_nodes = list(set(communities))
    
    # Find edges in the community network (slightly more complicated)
    community_edges = {} #Hash table because it makes collapsing edges easier
    internal_edges = [0] * num_communities #Counting num of links within each cluster

    for (i,j) in G.edges():
        
        c_i = communities[i]
        c_j = communities[j]
        
        w_ij = G.get_edge_data(i,j)['weight']
        
        if c_i != c_j:
            key = ' '.join([str(num) for num in sorted([c_i,c_j])]) # Example: '1 2'
            if key not in community_edges:
                community_edges[key] = w_ij
            else:
                community_edges[key] += w_ij
        else:
            internal_edges[c_i] += w_ij
    
    community_edges = [tuple([int(e) for e in k.split(" ")]+[v]) for (k,v) in community_edges.items()] # Horrible I know..
    
    G_communities.add_nodes_from(community_nodes)
    G_communities.add_weighted_edges_from(community_edges)
    
    
    ###############
    # BOILERPLATE #
    ###############
    
    pos = nx.spring_layout(G_communities, k=k_, iterations=10)
    
    # color map from http://colorbrewer2.org/
    cmapLight = colors.ListedColormap(['#a6cee3', '#b2df8a', '#fb9a99', '#fdbf6f', '#cab2d6'], 'indexed', num_communities)
    cmapDark = colors.ListedColormap(['#1f78b4', '#33a02c', '#e31a1c', '#ff7f00', '#6a3d9a'], 'indexed', num_communities)

    # edges
    weights_transformed = np.log(np.array([w for (i,j,w) in community_edges]))**3+100
    nx.draw_networkx_edges(G_communities, 
                           pos,
                           width=weights_transformed/max(weights_transformed)
    )

    # nodes
    nodeCollection = nx.draw_networkx_nodes(G_communities,
        pos = pos,
        node_color = G_communities.nodes(),
        cmap = cmapLight,
        node_size = ((np.array(internal_edges))*1.85/(np.array(internal_edges)).max()+0.15)*1000
    )
    # set node border color to the darker shade
    darkColors = [cmapDark(v) for v in G_communities.nodes()]
    nodeCollection.set_edgecolor(darkColors)
    
    # Node labels
    for n in G_communities.nodes_iter():
        plt.annotate(n,
            xy = pos[n],
            textcoords = 'offset points',
            horizontalalignment = 'center',
            verticalalignment = 'center',
            xytext = [0, 2],
            color = cmapDark(n)
        )

    plt.axis('off')
    #plt.savefig("netw.png")
    
    
#draw_multiplex_network_communities([layer1,layer2],k_=2)

In [29]:
import numpy as np

In [2]:
from graphtool import *

ImportError: No module named graph_tool